In [2]:
from PIL import Image
import numpy as np
import os
import cv2

In [3]:
data=[]
labels=[]

cats=os.listdir("small_train")
for cat in cats:
    imag=cv2.imread("small_train/"+cat)
    img_from_ar = Image.fromarray(imag, 'RGB')
    resized_image = img_from_ar.resize((50, 50))
    
    data.append(np.array(resized_image))
    labels.append(0)
    
dogs=os.listdir("preview")
for dog in dogs:
    imag=cv2.imread("preview/"+dog)
    img_from_ar = Image.fromarray(imag, 'RGB')
    resized_image = img_from_ar.resize((50, 50))
    
    data.append(np.array(resized_image))
    labels.append(1)

In [4]:
animals = np.array(data)
labels = np.array(labels)

In [35]:
np.save("animals",animals)
np.save("labels",labels)

In [36]:
s=np.arange(animals.shape[0])
np.random.shuffle(s)
animals=animals[s]
labels=labels[s]

In [37]:
num_classes=len(np.unique(labels))
data_length=len(animals)

In [38]:
(x_train,x_test)=animals[(int)(0.1*data_length):],animals[:(int)(0.1*data_length)]
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
train_length=len(x_train)
test_length=len(x_test)

In [39]:
(y_train,y_test)=labels[(int)(0.1*data_length):],labels[:(int)(0.1*data_length)]

In [40]:
import keras
from keras.utils import np_utils
#One hot encoding
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [41]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [29]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))

model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))

model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))

model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(500,activation="relu"))

model.add(Dropout(0.2))

model.add(Dense(2,activation="softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 6, 6, 64)          0         
__________

In [42]:
model.compile(loss='categorical_crossentropy', optimizer='adam', 
                  metrics=['accuracy'])

In [43]:
model.fit(x_train,y_train,batch_size=50
          ,epochs=100,verbose=1)

Epoch 1/100
72/72 [==============================] - 16s 220ms/step - loss: 0.7262 - acc: 0.5000
Epoch 2/100
72/72 [==============================] - 4s 49ms/step - loss: 0.6781 - acc: 0.7222
Epoch 3/100
72/72 [==============================] - 3s 48ms/step - loss: 0.6203 - acc: 0.7639
Epoch 4/100
72/72 [==============================] - 3s 48ms/step - loss: 0.6113 - acc: 0.7778
Epoch 5/100
72/72 [==============================] - 3s 48ms/step - loss: 0.5423 - acc: 0.7222
Epoch 6/100
72/72 [==============================] - 4s 49ms/step - loss: 0.5432 - acc: 0.7222
Epoch 7/100
72/72 [==============================] - 4s 49ms/step - loss: 0.4692 - acc: 0.7222
Epoch 8/100
72/72 [==============================] - 4s 50ms/step - loss: 0.3873 - acc: 0.7361
Epoch 9/100
72/72 [==============================] - 3s 48ms/step - loss: 0.3233 - acc: 0.8056
Epoch 10/100
72/72 [==============================] - 3s 47ms/step - loss: 0.2579 - acc: 0.9583
Epoch 11/100
72/72 [===========================

In [44]:
model.evaluate(x_test, y_test, verbose=1)

7/7 [==============================] - 0s 10ms/step


[1.5613833665847778, 0.8571428656578064]

In [45]:
def convert_to_array(img):
    im = cv2.imread(img)
    img = Image.fromarray(im, 'RGB')
    image = img.resize((50, 50))
    return np.array(image)

In [46]:
def get_animal_name(label):
    if label==0:
        return "cat"
    if label==1:
        return "dog"
    if label==2:
        return "bird"
    if label==3:
        return "fish"

In [47]:
def predict_animal(file):
    print("Predicting .................................")
    ar=convert_to_array(file)
    ar=ar/255
    label=1
    a=[]
    a.append(ar)
    a=np.array(a)
    score=model.predict(a,verbose=1)
    print(score)
    label_index=np.argmax(score)
    print(label_index)
    acc=np.max(score)
    animal=get_animal_name(label_index)
    print(animal)
    print("The predicted Animal is a "+animal+" with accuracy =    "+str(acc))

In [61]:
predict_animal("ranveersingh4.jpg")
# predict_animal("IMG_2243.png")

Predicting .................................
1/1 [==============================] - 0s 0us/step
[[0.88458717 0.11541288]]
0
cat
The predicted Animal is a cat with accuracy =    0.88458717
